In [ ]:
import contextlib

In [ ]:
@contextlib.contextmanager
def context():
    print('start')
    print('a')
    try:
        yield "abc"
    finally:
        print("exit")

In [ ]:
with context():
    print('haha')

In [ ]:
type(context)

In [ ]:
def add(x,y):
    yield x+y

In [ ]:
add(1,2).__name__

In [ ]:
class ContextManager:
    def __init__(self,fn):
        self.fn = fn
    def __call__(self,*args,**kwargs):
        self.gen = self.fn(*args,**kwargs)
        return self 
    def __enter__(self):
        try:
            return next(self.gen)
        except StopIteration:
            pass
    def __exit__(self,exc_type,exc,traceback):
        try:
            return next(self.gen)
        except StopIteration:
            pass
        return False

In [ ]:
@ContextManager
def context2(x,y):
    print('start')
    print('a')
    try:
        yield x+y
    finally:
        print("exit")

In [ ]:
with context2(1,2):
    print('haha')

In [1]:
import threading

In [2]:
def worker():
    print('work')

In [3]:
thread = threading.Thread(target=worker)

In [4]:
thread.start()

work


In [4]:
import threading
import logging
import importlib
importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')
class Task:
    def __init__(self,name):
        self.name = name
        self.lock = threading.Lock()
def worker(tasks):
    for task in tasks:
        if task.lock.acquire(False):
            logging.info(task.name)
tasks = [Task(x) for x in range(10)]
for x in range(5):
    threading.Thread(target=worker,args=(tasks,),name='worker-{}'.format(x))

In [5]:
logging.info('aaa')

2017-03-11 16:58:03,901 INFO [MainThread] aaa


In [17]:
import random
class Dispatcher:
    def __init__(self):
        self.data = None
        self.event = threading.Event()
        self.cond = threading.Condition()
    
    def consumer(self):
        while not self.event.is_set():
            with self.cond:
                self.cond.wait()
                logging.info(self.data)
    def producer(self):
        for _ in range(10):
            data = random.randint(0,100)
            logging.info(data)
            self.data = data
            with self.cond:
                self.cond.notify_all()
            self.event.wait(1)
        self.event.set()

In [18]:
d = Dispatcher()

In [19]:
c = threading.Thread(target=d.consumer,name='consumer')

In [20]:
p = threading.Thread(target=d.producer,name='producer')

In [21]:
c.start()

In [22]:
p.start()

2017-03-11 17:32:44,784 INFO [producer] 20
2017-03-11 17:32:44,786 INFO [consumer] 20
2017-03-11 17:32:45,787 INFO [producer] 18
2017-03-11 17:32:45,789 INFO [consumer] 18
2017-03-11 17:32:46,789 INFO [producer] 26
2017-03-11 17:32:46,791 INFO [consumer] 26
2017-03-11 17:32:47,791 INFO [producer] 83
2017-03-11 17:32:47,809 INFO [consumer] 83
2017-03-11 17:32:48,825 INFO [producer] 15
2017-03-11 17:32:48,830 INFO [consumer] 15
